# Basic validation of `grad2` and `counting number`

In [1]:
import drjit as dr
import mitsuba as mi
import numpy as np


mi.set_variant('llvm_ad_rgb')
# dr.set_flag(dr.JitFlag.LoopRecord, False)
# dr.set_flag(dr.JitFlag.VCallRecord, False)

# dr.set_log_level(dr.LogLevel.Trace)

In [2]:
dr.__file__

'/home/yanni/mitsuba3/build/python/drjit/__init__.py'

In [3]:
checkerboard = True
mirror =  not checkerboard

visualize_ad_graph = True


It should be noted that here we use directional light.

In [4]:
# the hdr film filter is set to box instead of gaussian
if checkerboard:
    scene = mi.load_file('scenes/rectangle.xml', spp = 1, resx = 2, resy = 2, max_depth = 6)
if mirror:
    scene = mi.load_file('scenes/mirror.xml', spp = 10, resx = 64, resy = 64, max_depth = 6)


In [5]:
image_ref = scene.integrator().render(scene, scene.sensors()[0])
mi.util.convert_to_bitmap(image_ref)


Bitmap[
  pixel_format = rgb,
  component_format = uint8,
  size = [2, 2],
  srgb_gamma = 1,
  struct = Struct<3>[
    uint8 R; // @0, normalized, gamma, premultiplied alpha
    uint8 G; // @1, normalized, gamma, premultiplied alpha
    uint8 B; // @2, normalized, gamma, premultiplied alpha
  ],
  data = [ 12 B of image data ]
]

### Reference image

In [6]:
# image_ref = mi.render(scene)
# # Preview the reference image
# mi.util.convert_to_bitmap(image_ref)

In [7]:
if checkerboard:
    mi.Bitmap(image_ref).write('22.exr')
    mi.util.write_bitmap('22.png', image_ref)
    print(image_ref.numpy())
if mirror:
    mi.Bitmap(image_ref).write('mirror.exr')
    mi.util.write_bitmap('mirror.png', image_ref)


[[[0.        0.        0.       ]
  [1.5915494 1.5915494 1.5915494]]

 [[1.5915494 1.5915494 1.5915494]
  [0.        0.        0.       ]]]


In [8]:
params = mi.traverse(scene)
# params

### Initial state

In [9]:
key = 'color_checkerboard.color1.value'
# key = 'red.reflectance.value'

# Save the original value
param_ref = mi.Color3f(params[key])

# Set another color value and update the scene
params[key] = mi.Color3f(0.01, 0.2, 0.9)
dr.enable_grad(params[key])
params.update();

As expected, when rendering the scene again, the wall has changed color.

In [10]:
image_init = scene.integrator().render(scene, scene.sensors()[0])
mi.util.convert_to_bitmap(image_init)

Bitmap[
  pixel_format = rgb,
  component_format = uint8,
  size = [2, 2],
  srgb_gamma = 1,
  struct = Struct<3>[
    uint8 R; // @0, normalized, gamma, premultiplied alpha
    uint8 G; // @1, normalized, gamma, premultiplied alpha
    uint8 B; // @2, normalized, gamma, premultiplied alpha
  ],
  data = [ 12 B of image data ]
]

In [11]:
# image_init = mi.render(scene)
# mi.util.convert_to_bitmap(image_init)

In [12]:
if checkerboard:
    mi.Bitmap(image_init).write('22_init.exr')
    mi.util.write_bitmap('22_init.png', image_init)
    print(image_init.numpy())
if mirror:
    mi.Bitmap(image_init).write('mirror_init.exr')
    mi.util.write_bitmap('mirror_init.png', image_init)

[[[0.01591549 0.3183099  1.4323944 ]
  [1.5915494  1.5915494  1.5915494 ]]

 [[1.5915494  1.5915494  1.5915494 ]
  [0.01591549 0.3183099  1.4323944 ]]]


In [13]:
def mse(image):
    return dr.mean(dr.sqr(image - image_ref))

#### 1. spp = 1, iteration = 10, random seed is set
\
Now we record the grads, grad2s, and counters for 10 iterations

In [14]:
grad = []
grad2 = []
counter = []
iter = 10

In [15]:
for i in range(iter):
    # it's important to set the seeds to get statistically independent samples
    image = mi.render(scene,params,seed=i,spp=1)      
    loss = mse(image)
    dr.backward(loss, flags = dr.ADFlag.BackPropVarianceCounter | dr.ADFlag.ClearVertices)
    g_ = dr.grad(params[key])
    g2_= dr.grad2(params[key])
    c_ = dr.counter(params[key])
    grad.append(g_)
    grad2.append(g2_)
    counter.append(c_)
    dr.set_grad(params[key], 0)

grad = np.array(grad, dtype=np.float64)
grad2 = np.array(grad2, dtype=np.float64)
counter = np.array(counter, dtype=np.float64)

As expected, the gradients( as well as grad2 and counter) are exactly the same between iterations

In [16]:
# print(grad2)
print(counter)

[[[2. 2. 2.]]

 [[2. 2. 2.]]

 [[2. 2. 2.]]

 [[2. 2. 2.]]

 [[2. 2. 2.]]

 [[2. 2. 2.]]

 [[2. 2. 2.]]

 [[2. 2. 2.]]

 [[2. 2. 2.]]

 [[2. 2. 2.]]]


In [17]:
m_grad = np.mean(grad, axis = 0,dtype=np.float64)
m_grad2 = np.mean(grad2, axis = 0,dtype=np.float64)
m_counter = np.mean(counter, axis = 0,dtype=np.float64)
# print(m_grad)
# print(m_grad2)
# print(m_counter)

#### 2. spp = 10, iteration = 1, random seed is set
Now we record the gradient for only one time

In [18]:
dr.set_grad(params[key], 0)
image = mi.render(scene, params,seed=iter+1, spp=iter)      
loss = mse(image)
# Backpropagate through the rendering process
dr.backward(loss, flags = dr.ADFlag.BackPropVarianceCounter | dr.ADFlag.ClearVertices)
g = dr.grad(params[key])
g2 = dr.grad2(params[key])
c = dr.counter(params[key])
g = np.array(g, dtype=np.float64)
g2 = np.array(g2, dtype=np.float64)
c = np.array(c, dtype=np.float64)
print(g)
print(g2)
print(c)

[[0.00844343 0.16886863 0.7599088 ]]
[[3.56457713e-06 1.42583088e-03 2.88730673e-02]]
[[20. 20. 20.]]


#### 3. Results
For spp = 1, iteration = N, and the scenaria where spp = N, iteration = 1, we find that
- The grad remains the same;
- The grad2 is N times smaller for spp = N
- The counter is N times bigger for spp = N

In [19]:
print(m_grad)
print(g)
print("-------------------------------------")
print(m_grad2)
print(g2)
print("-------------------------------------")
print(m_counter)
print(c)

[[0.00844343 0.16886865 0.75990874]]
[[0.00844343 0.16886863 0.7599088 ]]
-------------------------------------
[[3.56457713e-05 1.42583093e-02 2.88730651e-01]]
[[3.56457713e-06 1.42583088e-03 2.88730673e-02]]
-------------------------------------
[[2. 2. 2.]]
[[20. 20. 20.]]


The intuitive reasons are as follow:


| numbers | spp = 1 | result  | spp = N  | result  |
| :------------------- | :------------------- |:------------------- |:------------------- |:------------------- |
| edge weight | [1] | - | [1/N, 1/N, ..., 1/N], (1,N) | -|
| grad | [1 * grad] | grad | [1/N * grad, 1/N * grad, ..., 1/N *grad], (1,N) | grad|
| grad2 | [1 * 1 * grad2] | grad2 | [1/N * 1/N * grad2,  ..., 1/N * 1/N * grad2], (1,N) | 1/N * grad2|
| counter | [1 * counter] | counter | [1 * counter,  ..., 1 * counter], (1,N) | N * counter|

In [ ]:
if visualize_ad_graph:
    ad_graph = dr.graphviz_ad()
    ad_graph.view()
